In [2]:
#!pip install SimpleITK==2.0.2

In [16]:
import SimpleITK as sitk
import sys, glob, os
import pandas as pd
import shutil
import pprint, pickle
import numpy as np
from skimage import measure
import math
from skimage.measure import regionprops

In [21]:
prediction = 'F:/codes/ubuntu/nnUnet/3nnUnet/nnUNet_output/nnUNet_trained_models/nnUNet/3d_fullres/Task152_CellPress110/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/Task171_XJallTest/full/XJTr0001_full.nii.gz'
property_file = 'F:/codes/ubuntu/nnUnet/file/after_headcut_properties_XJdata.pkl'

In [7]:
def save_itk_from_numpy(numpy_data, property):

    pred_itk_image = sitk.GetImageFromArray(numpy_data)
    pred_itk_image.SetSpacing(property["itk_spacing"])
    pred_itk_image.SetOrigin(property["itk_origin"])

    return pred_itk_image

In [12]:
def compt_real_size(original_spacing):
    space_prod = np.prod(original_spacing)
    small_voxels = int(((4/3)*(math.pi)*((5/2)**3))/space_prod)
    middle_voxels = int(((4/3)*(math.pi)*((15/2)**3))/space_prod)
    return small_voxels,middle_voxels

In [9]:
def generate_datainfo_csv(name,small_IAs_num,middle_IAs_num,large_IAs_num):
 
    #name_attribute = ['id','small_IAs_num','middle_IAs_num','large_IAs_num']
    
    img = dict()
    img['id'] = name
    img['small_IAs_num'] = small_IAs_num
    img['middle_IAs_num'] = middle_IAs_num
    img['large_IAs_num'] = large_IAs_num
    
    return img

In [6]:
pkl_file = open(property_file, 'rb')
propertes = pickle.load(pkl_file)

In [23]:
import random
small_IAs_num, middle_IAs_num, large_IAs_num = 0,0,0

patientID = os.path.basename(prediction)[:-7]

pred_nii = sitk.ReadImage(prediction)
pred_np = sitk.GetArrayFromImage(pred_nii).astype(np.int32)

pred_bin_np = pred_np > 0
pred_lbls_np = measure.label(pred_bin_np)
labels = np.unique(pred_lbls_np)

if len(labels) == 1 and labels[0] == 0:
    print ('000')
    #shutil.copyfile(prediction,os.path.join(ori_folder,'small',patientID+'.nii.gz'))
    #shutil.copyfile(prediction,os.path.join(ori_folder,'middle',patientID+'.nii.gz'))
    #shutil.copyfile(prediction,os.path.join(ori_folder,'large',patientID+'.nii.gz'))

elif len(labels) > 1:
    labels = labels[1:]

    small_save_np = np.zeros_like(pred_lbls_np)
    middle_save_np = np.zeros_like(pred_lbls_np)
    large_save_np = np.zeros_like(pred_lbls_np)

    small_v,middle_v = compt_real_size(propertes[patientID[:-5]]['original_spacing'])

    for lbl in labels:
        pred_lbl_np = np.zeros_like(pred_lbls_np)
        lbl_voxels = np.sum(pred_lbls_np == lbl)
        assert lbl_voxels > 0
        
        print ('\n\n',lbl)
        pred_lbl_np[pred_lbls_np == lbl] = 1
        print (np.where(pred_lbl_np == 1))
        
        if lbl_voxels < small_v:
            small_save_np[pred_lbls_np == lbl] = 1
            small_IAs_num += 1
        elif (lbl_voxels >= small_v) and (lbl_voxels < middle_v):
            middle_save_np[pred_lbls_np == lbl] = 1
            middle_IAs_num += 1
        elif lbl_voxels >= middle_v:
            large_save_np[pred_lbls_np == lbl] = 1
            large_IAs_num += 1
    
    #small_nii = save_itk_from_numpy(small_save_np, propertes[patientID])
    #sitk.WriteImage(small_nii, os.path.join(ori_folder,'small',patientID+'.nii.gz'))

    #middle_nii = save_itk_from_numpy(middle_save_np, propertes[patientID])
    #sitk.WriteImage(middle_nii, os.path.join(ori_folder,'middle',patientID+'.nii.gz'))

    #large_nii = save_itk_from_numpy(large_save_np, propertes[patientID])
    #sitk.WriteImage(large_nii, os.path.join(ori_folder,'large',patientID+'.nii.gz'))
    
print (patientID,small_IAs_num, middle_IAs_num, large_IAs_num)
        
        



 1
(array([124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124,
       124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124,
       124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 124, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125,
       125, 125, 125, 125, 125, 125, 125, 125, 126, 126, 126, 126, 126,
       126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126,
       126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126,
       126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 126,
       126, 126, 126, 126, 126, 126, 126, 126, 126, 126, 1

In [19]:
name_attribute = ['id','small_IAs_num','middle_IAs_num','large_IAs_num']
save_csv(csv_info_list,save_csv_name,name_attribute)